생존여부 예측모델 만들기
학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)

In [434]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

1. 라이브러리 및 데이터 확인

In [435]:


def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    x_train, x_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = x_train[[id_name, target]]
    x_train = x_train.drop(columns=[target])

    
    y_test = x_test[[id_name, target]]
    x_test = x_test.drop(columns=[target])
    return x_train, x_test, y_train, y_test 
    
df = pd.read_csv("../../Datasat/T2-1/train.csv")
x_train, x_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

2. EDA

In [436]:
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

     PassengerId  Pclass                                       Name     Sex  \
90            91       3                       Christmann, Mr. Emil    male   
103          104       3                 Johansson, Mr. Gustaf Joel    male   
577          578       1  Silvey, Mrs. William Baird (Alice Munger)  female   

      Age  SibSp  Parch  Ticket     Fare Cabin Embarked  
90   29.0      0      0  343276   8.0500   NaN        S  
103  33.0      0      0    7540   8.6542   NaN        S  
577  39.0      1      0   13507  55.9000   E44        S  
     PassengerId  Pclass                           Name   Sex   Age  SibSp  \
210          211       3                 Ali, Mr. Ahmed  male  24.0      0   
876          877       3  Gustafsson, Mr. Alfred Ossian  male  20.0      0   
666          667       2    Butler, Mr. Reginald Fenton  male  25.0      0   

     Parch              Ticket     Fare Cabin Embarked  
210      0  SOTON/O.Q. 3101311   7.0500   NaN        S  
876      0              

In [437]:
print(x_train.info())
print(x_test.info())
print(y_train.info())


<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 90 to 116
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Name         712 non-null    object 
 3   Sex          712 non-null    object 
 4   Age          575 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Cabin        170 non-null    object 
 10  Embarked     711 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 210 to 45
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  179 non-null    int64  
 1   Pclass       179 non-null    int64  
 2   Name       

In [438]:
print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   712.000000  712.000000  575.000000  712.000000  712.000000  712.000000
mean    446.910112    2.285112   29.414783    0.533708    0.391854   33.388155
std     257.882594    0.842875   14.589601    1.099284    0.802311   50.807818
min       1.000000    1.000000    0.420000    0.000000    0.000000    0.000000
25%     226.750000    1.000000   20.000000    0.000000    0.000000    7.925000
50%     444.000000    3.000000   28.000000    0.000000    0.000000   15.047900
75%     672.250000    3.000000   37.000000    1.000000    0.000000   31.387500
max     891.000000    3.000000   74.000000    8.000000    6.000000  512.329200
       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   179.000000  179.000000  139.000000  179.000000  179.000000  179.000000
mean    442.379888    2.402235   30.875324    0.480447    0.340782   27.494878
std     255.926541    0.803920   14.253972    1.1184

In [439]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          542
Embarked         1
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             40
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          145
Embarked         1
dtype: int64
PassengerId    0
Survived       0
dtype: int64


# 결측치 대체
#x_train(712,11) : Age(137), Cabin(542), Embarked(1)
#x_test(179,11) : Age(40), Cabin(145), Embarked(1)

#변수제거
#(결측치 다수)Cabin

In [440]:
#생존 비율
y_train['Survived'].value_counts()

Survived
0    441
1    271
Name: count, dtype: int64

3. 데이터 전처리, 분리

In [441]:
x_train = x_train.drop(['Cabin','Name','Ticket'], axis=1)
x_test = x_test.drop(['Cabin','Name','Ticket'], axis=1)
print(x_train.info())
print(x_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 90 to 116
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Sex          712 non-null    object 
 3   Age          575 non-null    float64
 4   SibSp        712 non-null    int64  
 5   Parch        712 non-null    int64  
 6   Fare         712 non-null    float64
 7   Embarked     711 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 210 to 45
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  179 non-null    int64  
 1   Pclass       179 non-null    int64  
 2   Sex          179 non-null    object 
 3   Age          139 non-null    float64
 4   SibSp        179 non-null    int64  
 5   Parch        

In [442]:
# 결측치 대체
#x_train(712,11) : Age(137), Cabin(542), Embarked(1)
#x_test(179,11) : Age(40), Cabin(145), Embarked(1)

#Age 변수
median = x_train['Age'].median()
x_train['Age'] = x_train['Age'].fillna(median)
x_test['Age'] = x_test['Age'].fillna(median)

#Embarked 변수 -> object
mode_eb = x_train['Embarked'].mode()
x_train['Embarked'] = x_train['Embarked'].fillna(mode_eb[0])  #최빈값 [0]으로 해야함
x_test['Embarked'] = x_test['Embarked'].fillna(mode_eb[0])


In [443]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())


PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64


In [444]:
# 변수처리 (원핫인코딩)
# y= y_train["Survived"]

# features = ["Name", "Sex", "Ticket","Embarked"]
# x_train = pd.get_dummies(x_train[features])
# x_test = pd.get_dummies(x_test[features]) 
y = y_train["Survived"]

# sex만 원핫인코딩 됨
# features = ["Name", "Sex", "Ticket", "Embarked"]
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)


In [445]:
print(x_train.shape) 
print(x_test.shape)
# x.shape, test.shape

(712, 11)
(179, 11)


In [446]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['Survived'],test_size=0.2, stratify=y_train['Survived'], random_state=2023)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(569, 11)
(143, 11)
(569,)
(143,)


In [447]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2023)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=2023)

In [448]:
y_pred = model.predict(x_val)

In [449]:
model.score(x_train, y_train)

1.0

In [450]:
output = pd.DataFrame({'PassengerId': x_val.PassengerId, 'Survived': y_pred})
output.head()

,PassengerId,Survived
545,546,0
119,120,0
380,381,1
359,360,1
470,471,0
